In [1]:
from scipy.spatial import ConvexHull
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import copy
import numpy as np
import pickle
plt.rcParams.update({'font.size': 12})
import pandas as pd
from scipy.optimize import minimize
import csv

In [ ]:
node_x = []
for RVE_N in range(0,1200):
    NN = str(RVE_N+1)
    print('RVE',NN)
    file = '/scratch/bell/jbarsima/adip_size/tess_files/RVE_'+ NN +'.tess'
    #file = 'RVE_2.tess'
    data = []
    with open(file, 'r') as f:
        for line in f:
            line_data = line.strip().split()
            # append the data from this line to the list
            data.append(line_data)

    ## VERTEX DATA
    # loop through each element of the 'data' list
    for i in range(len(data)):
        # check if the current element matches the search value
        if data[i] == ['**vertex']:
            #print(f"Found '**vertex' at index {i}.")
            vertex_in = i #Index for vertex
            break

    n_vertex = int(data[vertex_in + 1][0]) #Number of vertices

    node_ref = np.zeros((1,3))
    for i in range(n_vertex):
        ii = i + vertex_in + 2
        x = float(data[ii][1])
        y = float(data[ii][2])
        z = float(data[ii][3])
        temp = np.array([x,y,z])
        node_ref = np.vstack((node_ref,temp))
    node_ref = node_ref[1:,:] #Vertices

    ## EDGE DATA
    # loop through each element of the 'data' list
    for i in range(n_vertex,len(data)):
        # check if the current element matches the search value
        if data[i] == ['**edge']:
            #print(f"Found '**edge' at index {i}.")
            edge_in = i #Index for edge

    n_edge = int(data[edge_in + 1][0]) #Number of edges

    edge_pos = np.zeros((1,2))
    for i in range(n_edge):
        ii = i + edge_in + 2
        ini_ = int(data[ii][1])
        end_ = int(data[ii][2])
        temp = np.array([ini_,end_])
        edge_pos = np.vstack((edge_pos,temp))
    edge_pos = edge_pos[1:,:] #edges

    ## FACE DATA
    # loop through each element of the 'data' list
    for i in range(n_edge + edge_in,len(data)):
        # check if the current element matches the search value
        if data[i] == ['**face']:
            #print(f"Found '**face' at index {i}.")
            face_in = i #Index for face
            break

    n_face = int(data[face_in + 1][0]) #Number of faces

    off = 2
    face_vert_f_all = []
    face_edge_f_all = []
    for i in range(n_face):
        ii = 4*i + face_in + off
        face_vert = data[ii] #Vertices that form the face
        face_edge = data[ii+1] #Edges that form the face
        face_vert_f = [] #Float
        face_edge_f = [] #Float
        for j in range(len(face_vert)):
            face_vert_f.append(float(face_vert[j]))
        face_vert_f = np.abs(face_vert_f[2:])
        # Array that containts all the vertices for the faces. Useful to find volume without
        # having to index edges
        face_vert_f_all.append(face_vert_f) 
        for m in range(len(face_edge)):
            face_edge_f.append(float(face_edge[m]))
        face_edge_f = np.abs(face_edge_f[1:])
        # Array that containts all the edges for the faces
        # Note that this has "the same" data as face_vert_f_all but 
        # with edge information instead of vertex information directly.
        # This one can be used to create a mesh as it lists the node
        # connectivity by edge. They are ordered correctly so they can be indexed.
        # For example, face 4 contains edges [4. 25. 26. 15. 27. 28.] which have nodes
        # [8. 2. 4. 1. 14. 15.], in that order. These edges can be used to index
        # the array edge_pos. For example, edge_pos[3] will give the nodes that make up
        # edge 4 (8 2). Then, using this data, one can index node_ref to obtain the coordinates
        # of the vertex. THIS PARTICULAR NUMBERS WERE OBTAINED FROM RVE_2
        face_edge_f_all.append(face_edge_f) 
    #print(face_edge_f_all[3])
    #print(face_vert_f_all[3])
    #print(edge_pos[3])
    #print(node_ref[7])

    ## POLYHEDRON DATA
    # loop through each element of the 'data' list
    for i in range(n_face + face_in,len(data)):
        # check if the current element matches the search value
        if data[i] == ['**polyhedron']:
            #print(f"Found '**polyhedron' at index {i}.")
            poly_in = i #Index for face
            break

    n_poly = int(data[poly_in + 1][0]) #Number of polyhedrons
    
    if n_poly >= 5000:
        print('Too many cells')
        continue
    
    node_x.append(node_ref)


RVE 1
RVE 2
RVE 3
RVE 4
Too many cells
RVE 5
Too many cells
RVE 6
Too many cells
RVE 7
RVE 8
RVE 9
RVE 10
RVE 11
RVE 12
RVE 13
RVE 14
RVE 15
RVE 16
RVE 17
RVE 18
RVE 19


In [ ]:
with open("node_x.dump" , "wb") as f:
     pickle.dump(node_x, f) 